In [1]:
# Import libraries and read data 
from urllib.request import urlopen
import json
import pandas as pd
import os

os.system("cd ..")
import plotly.express as px
# Read Data


In [2]:
print(os.listdir("./"))

#df = pd.read_csv("./dpc-covid19-ita-province.csv")
df = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv")

['popolazione.csv', 'limits_IT_provinces.geojson', 'makeMap.ipynb', 'total_cases.html', 'dpc-covid19-ita-province.csv', '.ipynb_checkpoints']


In [3]:
# Drop useless colums
df.drop(columns=['stato', 'codice_regione', 'denominazione_regione','sigla_provincia', 'lat', 'long', "note", "codice_nuts_1", "codice_nuts_2", "codice_nuts_3"], axis=1, inplace=True)
# Drop useless rows 
df = df[df['denominazione_provincia'] != 'In fase di definizione/aggiornamento']
df = df[df['denominazione_provincia'] != 'Fuori Regione / Provincia Autonoma']


# Columns manipulation
df = df.rename(columns={'codice_provincia': 'Province'})
df = df.rename(columns={'totale_casi': 'Totale Casi'})
df['Date'] = pd.to_datetime(df['data'], format="%Y-%m-%d")
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [4]:
df.head()

,data,Province,denominazione_provincia,Totale Casi,Date
0,2020-02-24T18:00:00,66,L'Aquila,0,2020-02-24
1,2020-02-24T18:00:00,67,Teramo,0,2020-02-24
2,2020-02-24T18:00:00,68,Pescara,0,2020-02-24
3,2020-02-24T18:00:00,69,Chieti,0,2020-02-24
5,2020-02-24T18:00:00,76,Potenza,0,2020-02-24


In [5]:
populations = pd.read_csv("./popolazione.csv")
populations["Popolazione"] = [n * 10**3 for n in populations["Popolazione"]]
iterator = zip(populations["Province / Regioni / Ripartizioni"], populations["Popolazione"])
populations = dict(iterator)
print(populations)

{'Torino': 2213000.0, 'Vercelli': 167200.0, 'Biella': 171800.0, 'Verbano-Cusio-Ossola': 155100.0, 'Novara': 362200.0, 'Cuneo': 582400.0, 'Asti': 209600.0, 'Alessandria': 411900.0, 'Piemonte': 4273200.0, 'Aosta': 123900.0, 'Varese': 879900.0, 'Como': 594700.0, 'Lecco': 332600.0, 'Sondrio': 179200.0, 'Milano': 3249800.0, 'Bergamo': 1099600.0, 'Brescia': 1247600.0, 'Pavia': 535000.0, 'Lodi': 225900.0, 'Cremona': 351700.0, 'Mantova': 403600.0, 'Monza e della Brianza': 867400.0, 'Lombardia': 9967000.0, 'Bolzano': 533700.0, 'Trento': 544700.0, 'Trentino-A. A.': 1078500.0, 'Verona': 922300.0, 'Vicenza': 850400.0, 'Belluno': 199600.0, 'Treviso': 878100.0, 'Venezia': 842900.0, 'Padova': 929500.0, 'Rovigo': 229700.0, 'Veneto': 4852500.0, 'Pordenone': 309100.0, 'Udine': 523400.0, 'Gorizia': 136800.0, 'Trieste': 229500.0, 'Friuli-V. G.': 1198800.0, 'Imperia': 208600.0, 'Savona': 268800.0, 'Genova': 816900.0, 'La Spezia': 215500.0, 'Liguria': 1509800.0, 'Piacenza': 284100.0, 'Parma': 453600.0, "Reg

In [ ]:
days = list(dict.fromkeys(df["Date"]))
provinces = list(dict.fromkeys(list(df["denominazione_provincia"])))
#province_keys = list(dict.fromkeys(list(df["Provincia"])))
incidences = pd.DataFrame(columns=["Date", "Province", "Name", "Inzidence"])

n = 7
i = 0
for (dayNow, dayOld) in zip(days[n:], days[:-n]):
    df_now = df[dayNow == df["Date"]]
    df_old = df[dayOld == df["Date"]]
    for name in provinces:
        population = populations[name]
        now_name = df_now[df_now["denominazione_provincia"] == name]
        cases_now = now_name["Totale Casi"].item()
        prov_code = now_name["Province"].item()
        cases_old = (df_old[df_old["denominazione_provincia"] == name])["Totale Casi"].item()
        incidence = (cases_now-cases_old)*10**5 / population
        line = [list(df_now["Date"])[0], prov_code, name, incidence]
        incidences.loc[i] = line
        i+=1
incidences.head()


In [ ]:
print(len(df), len(incidences)+7*107)

In [ ]:
fig = px.choropleth(df,
                    geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
                    locations='Province',
                    color='Totale Casi',
                    color_continuous_scale='Reds',
                    featureidkey='properties.prov_istat_code_num',
                    animation_frame='Date',
                    range_color=(0, max(df['Totale Casi'])))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html('total_cases.html')

In [ ]:
fig = px.choropleth(incidences,
                    geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
                    locations='Province',
                    color='Inzidence',
                    color_continuous_scale='Reds',
                    featureidkey='properties.prov_istat_code_num',
                    animation_frame='Date',
                    range_color=(0, 300))
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_geos(fitbounds="locations", visible=False)
fig.write_html('../index.html')